In [2]:
import pandas as pd
import numpy as np

WINTER_CSV_PATH = r"C:\Users\TalPNB22\OneDrive\Documents\University\Master\Ships\routes expirements from noa\winter_data_final.csv"

df = pd.read_csv(WINTER_CSV_PATH)
df.head()

,Season,Name,Replicate,Place,Sampling Date,Body Size (cm),Temp 0,Temp 1,Temp 2,Temp 3,...,Lived 33,Lived 34,Lived 35,Lived 36,Lived 37,Lived 38,Lived 39,Lived 40,Lived 41,Lived 42
0,winter,PISCES,a,Ashdod,21.3.24,4.0,22,9,17.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,winter,PISCES,b,Ashdod,21.3.24,9.0,22,9,17.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,winter,PISCES,c,Ashdod,21.3.24,5.0,22,9,17.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,winter,ELPIDA,a,Ashdod,21.3.24,7.5,22,9,9.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,winter,ELPIDA,b,Ashdod,21.3.24,8.0,22,9,9.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Fix lived columns

lived_columns = sorted([col for col in df.columns if 'Lived' in col], key=lambda x: int(x.split()[1]))
temperature_columns = sorted([col for col in df.columns if 'Temp' in col], key=lambda x: int(x.split()[1]))
salinity_columns = sorted([col for col in df.columns if 'Salinity' in col], key=lambda x: int(x.split()[1]))

dying_day = {}
# Iterate over each row
for idx, row in df.iterrows():
    # Find the last column containing 1
    last_col_with_one = None
    for col in lived_columns:
        if row[col] == 1:
            last_col_with_one = col

    # Fill all previous columns with 1
    last_col_with_one_index = lived_columns.index(last_col_with_one)
    df.loc[idx, lived_columns[:last_col_with_one_index + 1]] = 1

    first_col_with_zero_index = last_col_with_one_index + 1
    if first_col_with_zero_index < len(lived_columns) and df.loc[idx, lived_columns[first_col_with_zero_index]] == 0:
        # Assign nan to all following columns
        df.loc[idx, lived_columns[first_col_with_zero_index + 1:]] = np.nan
        df.loc[idx, temperature_columns[first_col_with_zero_index + 1:]] = np.nan
        df.loc[idx, salinity_columns[first_col_with_zero_index + 1:]] = np.nan

        dying_day[idx] = first_col_with_zero_index

df['dying_day'] = df.index.map(dying_day)

In [8]:
df.to_csv(r"C:\Users\TalPNB22\OneDrive\Documents\University\Master\Ships\routes expirements from noa\winter_data_fixed.csv", index=False)